In [15]:
import pandas as pd
import numpy as np
import dask
import dask.dataframe as dd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV

In [16]:
df = dd.read_csv('9_23_hotness_01_percent.csv', assume_missing = True, blocksize = 64e6)

In [17]:
df.head()

,GUID,SESSION_SKEY,SITE_ID,SESSION_START_DT,ITEM_ID,MIN_SEQNUM,MIN_EVENT_TIMESTAMP,USER_ID,PLATFORM,asp_usd,...,CATEG_GMB_P1D_BEFORE_VI,TTL_BI_P1D_BEFORE_VI,TTL_GMB_P1D_BEFORE_VI,ITEM_BI_SHARE_IN_CATEG_P1D_BEFORE_VI,ITEM_GMB_SHARE_IN_CATEG_P1D_BEFORE_VI,CATEG_BI_SHARE_IN_TTL_P1D_BEFORE_VI,CATEG_GMB_SHARE_IN_TTL_P1D_BEFORE_VI,BI_INVERSE_FREQUENCY_SCORE,GMB_INVERSE_FREQUENCY_SCORE,rnk
0,9937d3881710a6e6f02cd2e4ff7ad788,3.805118e+13,3.0,2020-07-30,1.311577e+11,43.0,2020-07-30 17:15:51,0.000000e+00,Browser: mWeb,5.0,...,1.538424e+05,6525144.0,2.188364e+08,0.00017,0.000218,0.001804,0.000703,0.001074,0.001582,0.006
1,d4f39376166e79924e73a830012f8ea5,3.805600e+13,0.0,2020-08-05,3.131665e+11,608.0,2020-08-05 07:18:45,1.840615e+09,Apps: iPhone,60.0,...,4.122317e+05,6624528.0,2.202547e+08,NaN,NaN,0.001694,0.001872,NaN,NaN,0.006
2,ace49b041640a9c0b93ba28affff02e4,3.804762e+13,0.0,2020-07-26,4.023208e+11,31.0,2020-07-26 14:20:40,0.000000e+00,Browser: Core site,503.0,...,1.881010e+07,6174417.0,1.871884e+08,NaN,NaN,0.048301,0.100488,NaN,NaN,0.006
3,8891830716d4b9474906df10011cd54d,3.805159e+13,3.0,2020-07-31,1.743708e+11,2454.0,2020-07-31 06:52:23,1.123359e+09,Apps: iPhone,46.0,...,1.332561e+06,6367867.0,2.133461e+08,NaN,NaN,0.021025,0.006246,NaN,NaN,0.006
4,963d16581730aad74a104d04ffa8f650,3.804917e+13,3.0,2020-07-28,3.016997e+11,1.0,2020-07-28 09:24:12,0.000000e+00,Browser: Core site,18.0,...,8.696573e+04,7063671.0,2.377354e+08,NaN,NaN,0.000338,0.000366,NaN,NaN,0.006


In [18]:
# quickly filling out NaNs with zeroes, otherwise astype() won't work
df = df.fillna(0)

In [19]:
# df.columns.to_list()

# dropping unused columns (drop time columns too: SESSION_START_DT, MIN_EVENT_TIMESTAMP, SESSION_MIN_TIMESTAMP):
to_drop =['ITEM_ID', 'GUID','SESSION_SKEY','SITE_ID', 'MIN_SEQNUM', 'MIN_SEQNUM', 'USER_ID', 'SEQNUM_RANK', 'show_cnt', 'rnk', 'SELLER_ID']
df = df.drop(columns = to_drop, axis = 1)

In [23]:
df['FREE_SHIPPING_YN'].head()

0    1.0
1    1.0
2    1.0
3    0.0
4    0.0
Name: FREE_SHIPPING_YN, dtype: float64

In [29]:
# df['stp_yn'].value_counts().compute()
df['stp_yn'].unique().compute()

0    0.0
1    1.0
Name: stp_yn, dtype: float64

In [7]:
# Changing columns' types to make the dataframe more performant
df.dtypes.to_dict()

type_mapping = {'PLATFORM': 'category', 'stp_yn': 'int8', 'map_yn': 'int8', 'mdm_yn': 'int8', 'sme_type_dt': 'int8', 'vi_coupon_show': 'int8',\
               'vp_yn': 'int8', 'active_item_yn': 'int8', 'return_accept_yn': 'int8', 'return_duration_in_vi': 'int8',\
               'qty_available': 'int16', 'qty_sold': 'int16', 'show_view_1hr_cnt': 'int16', 'show_view_24hr_cnt': 'int16',\
               'show_watch_1hr_cnt': 'int16', 'show_watch_24hr_cnt': 'int16','show_sld_1hr_cnt': 'int16', 'show_sld_24hr_cnt': 'int16',\
               'show_last_item_cnt': 'int16', 'show_limited_time_cnt': 'int16', 'show_limited_qty_cnt': 'int16',\
               'BIN_CNT': 'int16', 'BID_CNT': 'int16','OFFER_CNT': 'int16', 'WATCH_CNT': 'int16', 'ASQ_CNT': 'int16',\
               'BBO_CNT': 'int16', 'BBOC_CNT': 'int16', 'CONV_YN': 'int8', 'MQ_YN': 'int8', 'FP_YN': 'int8', 'BO_YN': 'int8',\
               'FREE_SHIPPING_YN': 'int8', 'ITEM_CONDITION': 'category', 'ITEM_CONDITION_DETAIL': 'category', 'META_CATEG_NAME': 'category',\
               'CATEG_LVL2_NAME': 'category','VERTICAL': 'category', 'RETURN_ACCPT_YN_NEW': 'int8', 'RETURN_TYPE': 'category', 'PHOTO_CNT': 'int8',\
               'AUCT_START_DT': 'datetime64[ns]', 'LIST_AGE_DAY': 'int16', 'BUYER_SEGMENT': 'category', 'BUYER_SEC_FROM_SESSION_START_TO_VI': 'int16',\
               'BUYER_CNTRY': 'category', 'SITE_NAME': 'category', 'SELLER_SEGMENT': 'category', 'SELLER_ETRS_YN':'int8',\
               'SELLER_CNTRY': 'category','TRX_DOMESTIC_YN': 'int8'}
df = df.astype(type_mapping)

# 'SELLER_GREATER_CHINA_YN': 'int8' --> make category

In [ ]:
for col in type_mapping.keys():
    df[col].unique().compute()

In [8]:
# handle times separately: SESSION_START_DT, MIN_EVENT_TIMESTAMP, SESSION_MIN_TIMESTAMP
type_to_date = ['SESSION_START_DT', 'MIN_EVENT_TIMESTAMP', 'SESSION_MIN_TIMESTAMP']
df[type_to_date] = df[type_to_date].apply(pd.to_datetime, axis = 1, errors = 'ignore',\
                                         meta={'SESSION_START_DT': 'object', 'MIN_EVENT_TIMESTAMP': 'object', 'SESSION_MIN_TIMESTAMP': 'object'})

In [9]:
df.dtypes.to_dict()

{'SESSION_START_DT': dtype('O'),
 'MIN_EVENT_TIMESTAMP': dtype('O'),
 'PLATFORM': CategoricalDtype(categories=['__UNKNOWN_CATEGORIES__'], ordered=False),
 'asp_usd': dtype('float64'),
 'stp_yn': dtype('int8'),
 'map_yn': dtype('int8'),
 'mdm_yn': dtype('int8'),
 'sme_type_dt': dtype('int8'),
 'vi_coupon_show': dtype('int8'),
 'vp_yn': dtype('int8'),
 'VP_PILL_CNT': dtype('float64'),
 'active_item_yn': dtype('int8'),
 'return_accept_yn': dtype('int8'),
 'return_duration_in_vi': dtype('int8'),
 'qty_available': dtype('int16'),
 'qty_sold': dtype('int16'),
 'show_view_1hr_cnt': dtype('int16'),
 'show_view_24hr_cnt': dtype('int16'),
 'show_watch_1hr_cnt': dtype('int16'),
 'show_watch_24hr_cnt': dtype('int16'),
 'show_sld_1hr_cnt': dtype('int16'),
 'show_sld_24hr_cnt': dtype('int16'),
 'show_last_item_cnt': dtype('int16'),
 'show_limited_time_cnt': dtype('int16'),
 'show_limited_qty_cnt': dtype('int16'),
 'BIN_CNT': dtype('int16'),
 'BID_CNT': dtype('int16'),
 'OFFER_CNT': dtype('int16'),
 

In [12]:
df.head()

ValueError: invalid literal for int() with base 10: 'TOP RATED (eTRS)'

In [ ]:
# Checking cardinality of categorical variables
categorical_vars = df.select_dtype(include = 'category')

"""
os_mask = [2,1,3,4]
df2['new_os'] = df2['OperatingSystems'].apply(lambda x: 'other' if x not in os_mask else x)

df2['new_os'].loc[~df2['OperatingSystems'].isin(os_mask)].unique() # checking that it was applied to the right rows
"""

In [ ]:
# checking NaNs once again

# checking normality of some of the continious features



# for categories: dummies
# for cont: normalization 

categorical_vars = df.select_dtype(include = 'category')
numeric_vars = df.select_dtype(exclude = ['category', 'object'])

categorical_transformer = Pipeline(steps = [('onehot', OneHotEncoder(handle_unknown = 'ignore'))])
numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')), ('scaler', StandardScaler()),])

# Combine the result
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_vars),
        ('cat', categorical_transformer, categorical_vars)
    ]
)

# the main pipe
clf = Pipeline([('preprocessor', preprocessor), ('clf', RandomForestClassifier)])

# train/test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf.fit(X_train, y_train)